In [1]:
import sys
import os
import numpy as np
import re
import logging
import json

In [2]:
sys.path.append('./')
%env CUDA_VICIBLE_DEVICES=0
# if CUDA_VICIBLE_DEVICES is changed, relaunch jupyter kernel to inform paddle

env: CUDA_VICIBLE_DEVICES=0


In [3]:
import propeller.paddle as propeller
import paddle
import paddle.fluid as F
import paddle.fluid.layers as L
#import model defenition from original ERNIE
from model.ernie import ErnieModel
from tokenization import FullTokenizer
from optimization import optimization
from propeller import log
log.setLevel(logging.DEBUG)

if paddle.__version__ not in ['1.5.1', '1.5.2']:
    raise RuntimeError('propeller works in paddle1.5.1')


In [4]:
%%bash
# download pretrained model&config(ernie1.0) and xnli data
mkdir ernie1.0_pretrained
if [ ! -f ernie1.0_pretrained/ERNIE_stable-1.0.1.tar.gz ]
then
    echo "download model"
    wget --no-check-certificate https://baidu-nlp.bj.bcebos.com/ERNIE_stable-1.0.1.tar.gz -P ernie1.0_pretrained
fi

if [ ! -f task_data_zh.tgz ]
then
    echo "download data"
    wget --no-check-certificate https://ernie.bj.bcebos.com/task_data_zh.tgz
fi

tar xzf ernie1.0_pretrained/ERNIE_stable-1.0.1.tar.gz -C ernie1.0_pretrained
tar xzf task_data_zh.tgz

mkdir: cannot create directory ‘ernie1.0_pretrained’: File exists


In [5]:
#define basic training settings
EPOCH=3
BATCH=16
LR=5e-3
MAX_SEQLEN=128
TASK_DATA='./task_data/'
MODEL='./ernie1.0_pretrained/'
OUTPUT_DIR='./output'

In [6]:
!rm -rf {OUTPUT_DIR}

In [7]:
#skip header, and reorganize train data into ./xnli_data 
!mkdir xnli_data
!mkdir xnli_data/train
!mkdir xnli_data/test
!mkdir xnli_data/dev

def remove_header_and_save(fname_in, fname_out):
    with open(fname_out, 'w') as fout:
        buf = open(fname_in).readlines()[1:]
        for i in buf:
            fout.write(i)
        return len(buf)
train_data_size = remove_header_and_save(TASK_DATA + '/xnli/train.tsv', './xnli_data/train/part.0') 
dev_data_size = remove_header_and_save(TASK_DATA + '/xnli/dev.tsv', './xnli_data/dev/part.0') 
test_data_size = remove_header_and_save(TASK_DATA + '/xnli/test.tsv', './xnli_data/test/part.0') 
print(train_data_size)
print(dev_data_size)
print(test_data_size)


mkdir: cannot create directory ‘xnli_data’: File exists
mkdir: cannot create directory ‘xnli_data/train’: File exists
mkdir: cannot create directory ‘xnli_data/test’: File exists
mkdir: cannot create directory ‘xnli_data/dev’: File exists
392702
2490
5010


In [8]:
tokenizer = FullTokenizer(MODEL + 'vocab.txt')
vocab = {j.strip().split('\t')[0]: i for i, j in enumerate(open(MODEL + 'vocab.txt', encoding='utf8'))}

print(tokenizer.tokenize('今天很热'))
print(tokenizer.tokenize('coding in paddle is cool'))
print(tokenizer.tokenize('[CLS]i have an pen')) # note: special token like [CLS], will be segmented, so please add these id after tokenization.


['今', '天', '很', '热']
['cod', '##ing', 'in', 'pad', '##d', '##le', 'is', 'cool']
['[', 'cl', '##s', '[UNK]', 'i', 'have', 'an', 'pen']


`propeller.data.FeatureColumns` defines the data schema in every data file.

our data consist of 3 columns: seg_a, seg_b, label. with "\t" as delemeter.

`TextColumn` will do 3 things for you: 

1. tokenize input sentence with user-defined `tokenizer_func`
2. vocab lookup
3. serialize to protobuf bin file (optional)

data file is organized into following patten:

```script
./xnli_data
|-- dev
|   `-- part.0
|-- test
|   `-- part.0
|-- train
   `-- part.0
```

In [9]:

sep_id = vocab['[SEP]']
cls_id = vocab['[CLS]']
unk_id = vocab['[UNK]']

label_map = {
    b"contradictory": 0,
    b"contradiction": 0,
    b"entailment": 1,
    b"neutral": 2,
}
def tokenizer_func(inputs):
    ret = tokenizer.tokenize(inputs) #`tokenize` will conver bytes to str, so we use a str vocab
    return ret

feature_column = propeller.data.FeatureColumns([
    propeller.data.TextColumn('title', unk_id=unk_id, vocab_dict=vocab, tokenizer=tokenizer_func),
    propeller.data.TextColumn('comment', unk_id=unk_id, vocab_dict=vocab, tokenizer=tokenizer_func),
    propeller.data.LabelColumn('label', vocab_dict=label_map), #be careful, Columns deal with python3 bytes directly.
])

## trian model in propeller can be defined in 2 ways:
1. subclass of `propeller.train.Model` which implements:
    1. `__init__`           (hyper_param, mode, run_config)
    2. `forward`            (features) => (prediction)
    3. `backword`           (loss) => None
    4. `loss`               (predictoin) => (loss)
    5. `metrics` (optional) (prediction) => (dict of propeller.Metrics)
    
2. a callable takes following args:
    1. features
    2. param
    3. mode
    4. run_config(optional)
    
   and returns a propeller.ModelSpec
   
we use the subclasss approch here

In [10]:
class ClassificationErnieModel(propeller.train.Model):
    def __init__(self, hparam, mode, run_config):
        self.hparam = hparam
        self.mode = mode
        self.run_config = run_config

    def forward(self, features):
        src_ids, sent_ids = features
        dtype = 'float16' if self.hparam['fp16'] else 'float32'
        zero = L.fill_constant([1], dtype='int64', value=0)
        input_mask = L.cast(L.equal(src_ids, zero), dtype) # assume pad id == 0
        #input_mask = L.unsqueeze(input_mask, axes=[2])
        d_shape = L.shape(src_ids)
        seqlen = d_shape[1]
        batch_size = d_shape[0]
        pos_ids = L.unsqueeze(L.range(0, seqlen, 1, dtype='int32'), axes=[0])
        pos_ids = L.expand(pos_ids, [batch_size, 1])
        pos_ids = L.unsqueeze(pos_ids, axes=[2])
        pos_ids = L.cast(pos_ids, 'int64')
        pos_ids.stop_gradient = True
        input_mask.stop_gradient = True
        task_ids = L.zeros_like(src_ids) + self.hparam.task_id #this shit wont use at the moment
        task_ids.stop_gradient = True

        ernie = ErnieModel(
            src_ids=src_ids,
            position_ids=pos_ids,
            sentence_ids=sent_ids,
            task_ids=task_ids,
            input_mask=input_mask,
            config=self.hparam,
            use_fp16=self.hparam['fp16']
        )

        cls_feats = ernie.get_pooled_output()

        cls_feats = L.dropout(
            x=cls_feats,
            dropout_prob=0.1,
            dropout_implementation="upscale_in_train"
        )

        logits = L.fc(
            input=cls_feats,
            size=self.hparam['num_label'],
            param_attr=F.ParamAttr(
                name="cls_out_w",
                initializer=F.initializer.TruncatedNormal(scale=0.02)),
            bias_attr=F.ParamAttr(
                name="cls_out_b", initializer=F.initializer.Constant(0.))
        )

        propeller.summary.histogram('pred', logits)

        if self.mode is propeller.RunMode.PREDICT:
            probs = L.softmax(logits)
            return probs
        else:
            return logits

    def loss(self, predictions, labels):
        ce_loss, probs = L.softmax_with_cross_entropy(
            logits=predictions, label=labels, return_softmax=True)
        #L.Print(ce_loss, message='per_example_loss')
        loss = L.mean(x=ce_loss)
        return loss

    def backward(self, loss):
        scheduled_lr, loss_scale = optimization(
            loss=loss,
            warmup_steps=int(self.run_config.max_steps * self.hparam['warmup_proportion']),
            num_train_steps=self.run_config.max_steps,
            learning_rate=self.hparam['learning_rate'],
            train_program=F.default_main_program(),
            startup_prog=F.default_startup_program(),
            weight_decay=self.hparam['weight_decay'],
            scheduler="linear_warmup_decay",)
        propeller.summary.scalar('lr', scheduled_lr)

    def metrics(self, predictions, label):
        predictions = L.argmax(predictions, axis=1)
        predictions = L.unsqueeze(predictions, axes=[1])
        acc = propeller.metrics.Acc(label, predictions)
        #auc = propeller.metrics.Auc(label, predictions)
        return {'acc': acc}
    

In [11]:
# define some utility function.

def build_2_pair(seg_a, seg_b):
    token_type_a = np.ones_like(seg_a, dtype=np.int64) * 0
    token_type_b = np.ones_like(seg_b, dtype=np.int64) * 1
    sen_emb = np.concatenate([[cls_id], seg_a, [sep_id], seg_b, [sep_id]], 0)
    token_type_emb = np.concatenate([[0], token_type_a, [0], token_type_b, [1]], 0)
    #seqlen = sen_emb.shape[0]
    #deteministic truncate
    sen_emb = sen_emb[0: MAX_SEQLEN]
    token_type_emb = token_type_emb[0: MAX_SEQLEN]
    return sen_emb, token_type_emb

def expand_dims(*args):
    func = lambda i: np.expand_dims(i, -1)
    ret = [func(i) for i in args]
    return ret

def before_pad(seg_a, seg_b, label):
    sentence, segments = build_2_pair(seg_a, seg_b)
    return sentence, segments, label

def after_pad(sentence, segments, label):
    sentence, segments, label = expand_dims(sentence, segments, label)
    return sentence, segments, label

In [12]:
# a `propeller.paddle.data.Dataset` is built from FeatureColumns

train_ds = feature_column.build_dataset('train', use_gz=False, data_dir='./xnli_data/train', shuffle=True, repeat=True) \
                               .map(before_pad) \
                               .padded_batch(BATCH, (0, 0, 0)) \
                               .map(after_pad)

dev_ds = feature_column.build_dataset('dev', use_gz=False, data_dir='./xnli_data/dev', shuffle=False, repeat=False) \
                               .map(before_pad) \
                               .padded_batch(BATCH, (0, 0, 0)) \
                               .map(after_pad)

shapes = ([-1, MAX_SEQLEN, 1], [-1, MAX_SEQLEN, 1], [-1, 1])
types = ('int64', 'int64', 'int64')
train_ds.data_shapes = shapes
train_ds.data_types = types
dev_ds.data_shapes = shapes
dev_ds.data_types = types

warm_start_dir = MODEL + '/params'
# only the encoder and embedding is loaded from pretrained model
varname_to_warmstart = re.compile('^encoder.*w_0$|^encoder.*b_0$|^.*embedding$|^.*bias$|^.*scale$')
ws = propeller.WarmStartSetting(
        predicate_fn=lambda v: varname_to_warmstart.match(v.name) and os.path.exists(os.path.join(warm_start_dir, v.name)),
        from_dir=warm_start_dir
    )

# propeller will export model of highest performance, the criteria is up to you. 
# here we pick the model with maximum evaluatoin accuracy.
#`BestInferenceModelExporter` is used to export serveable models
best_inference_exporter = propeller.train.exporter.BestInferenceModelExporter(
    os.path.join(OUTPUT_DIR, 'best'), 
    cmp_fn=lambda old, new: new['eval']['acc'] > old['eval']['acc'])
#`BestExporter` is used to export restartable checkpoint, so that we can restore from it and check test-set accuracy.
best_exporter = propeller.train.exporter.BestExporter(
    os.path.join(OUTPUT_DIR, 'best_model'), 
    cmp_fn=lambda old, new: new['eval']['acc'] > old['eval']['acc'])


[INFO] 2019-09-18 16:26:51,230 [feature_column.py:  266]:	reading raw files from ./xnli_data/train/part.0
[INFO] 2019-09-18 16:26:51,232 [feature_column.py:  266]:	reading raw files from ./xnli_data/dev/part.0


In [13]:
#ERNIE1.0 config 
ernie_config = propeller.HParams(**json.loads(open(MODEL + '/ernie_config.json').read()))

# default term in official config
ernie_v2_config = propeller.HParams(**{
    "sent_type_vocab_size": None,  
    "use_task_id": False,
    "task_id": 0,
})

# train schema
train_config = propeller.HParams(**{  
      "warmup_proportion":  0.1,
      "weight_decay": 0.01,
      "fp16": 0,
      "learning_rate": 0.00005,
      "num_label": 3,
      "batch_size": 32
})

config = ernie_config.join(ernie_v2_config).join(train_config)

run_config = propeller.RunConfig(
    model_dir=OUTPUT_DIR,
    max_steps=EPOCH * train_data_size / BATCH,
    skip_steps=10,
    eval_steps=1000,
    save_steps=1000,
    log_steps=10,
    max_ckpt=3
)
            

## Finetune and Eval

In [14]:
# `train_and_eval` takes key-word args only
# we are now ready to train
hooks = [propeller.train.TqdmNotebookProgressBarHook(run_config.max_steps)] # to show the progress bar, you need to `pip install tqdm ipywidgets`
propeller.train_and_eval(
    model_class_or_model_fn=ClassificationErnieModel, #**careful**, you should pass a Class to `train_and_eval`, propeller will try to instantiate it.
    params=config, 
    run_config=run_config, 
    train_dataset=train_ds, 
    eval_dataset=dev_ds, 
    warm_start_setting=ws, 
    exporters=[best_exporter, best_inference_exporter],
    train_hooks=hooks,
)

[INFO] 2019-09-18 16:26:51,288 [  trainer.py:  218]:	Building Eval Graph
[INFO] 2019-09-18 16:26:51,889 [  trainer.py:  222]:	Done
[INFO] 2019-09-18 16:26:51,925 [  trainer.py:  226]:	Eval with: 
> Run_config: RunConfig(model_dir='./output', run_steps=None, max_steps=73631.625, save_steps=1000, eval_steps=1000, eval_max_steps=None, skip_steps=10, log_steps=10, max_ckpt=3, shit=None)
> Params: {'attention_probs_dropout_prob': 0.1, 'hidden_act': 'relu', 'hidden_dropout_prob': 0.1, 'hidden_size': 768, 'initializer_range': 0.02, 'max_position_embeddings': 513, 'num_attention_heads': 12, 'num_hidden_layers': 12, 'type_vocab_size': 2, 'vocab_size': 18000, 'sent_type_vocab_size': None, 'use_task_id': False, 'task_id': 0, 'warmup_proportion': 0.1, 'weight_decay': 0.01, 'fp16': 0, 'learning_rate': 5e-05, 'num_label': 3, 'batch_size': 32}
> Train_model_spec: ModelSpec(loss=name: "mean_0.tmp_0"
type {
  type: LOD_TENSOR
  lod_tensor {
    tensor {
      data_type: FP32
      dims: 1
    }
  }
}
p

[DEBUG] 2019-09-18 16:26:58,766 [monitored_executor.py:  281]:	freezing program
[INFO] 2019-09-18 16:26:58,767 [monitored_executor.py:  269]:	replica id 0 of 1
     You can try our memory optimize feature to save your memory usage:
         # create a build_strategy variable to set memory optimize option
         build_strategy = compiler.BuildStrategy()
         build_strategy.enable_inplace = True
         build_strategy.memory_optimize = True
         
         # pass the build_strategy to with_data_parallel API
         compiled_prog = compiler.CompiledProgram(main).with_data_parallel(
             loss_name=loss.name, build_strategy=build_strategy)
      
     !!! Memory optimize is our experimental feature !!!
         some variables may be removed/reused internal to save memory usage, 
         in order to fetch the right value of the fetch_list, please set the 
         persistable property to true for each variable in fetch_list

         # Sample
         conv1 = fluid.layers

[DEBUG] 2019-09-18 16:35:46,824 [    hooks.py:  207]:	step: 500	steps/sec: 1.09231	loss: 1.09590	lr:[3.395355e-06]
[DEBUG] 2019-09-18 16:35:56,298 [    hooks.py:  207]:	step: 510	steps/sec: 1.05217	loss: 1.05729	lr:[3.4632621e-06]
[DEBUG] 2019-09-18 16:36:05,206 [    hooks.py:  207]:	step: 520	steps/sec: 1.07546	loss: 1.08980	lr:[3.531169e-06]
[DEBUG] 2019-09-18 16:36:15,748 [    hooks.py:  207]:	step: 530	steps/sec: 0.96530	loss: 1.10138	lr:[3.5990763e-06]
[DEBUG] 2019-09-18 16:36:23,542 [    hooks.py:  207]:	step: 540	steps/sec: 1.22841	loss: 1.01145	lr:[3.6669835e-06]
[DEBUG] 2019-09-18 16:36:32,876 [    hooks.py:  207]:	step: 550	steps/sec: 1.06315	loss: 1.10759	lr:[3.7348907e-06]
[DEBUG] 2019-09-18 16:36:40,753 [    hooks.py:  207]:	step: 560	steps/sec: 1.40666	loss: 0.98144	lr:[3.8027974e-06]
[DEBUG] 2019-09-18 16:36:47,344 [    hooks.py:  207]:	step: 570	steps/sec: 1.44630	loss: 1.06504	lr:[3.870705e-06]
[DEBUG] 2019-09-18 16:36:57,775 [    hooks.py:  207]:	step: 580	steps/sec: 

[DEBUG] 2019-09-18 16:50:52,067 [ exporter.py:   75]:	New evaluate result: {'eval': {'acc': 0.6112449799196787, 'loss': 0.8885895720659158}} 
old: None
[DEBUG] 2019-09-18 16:50:52,067 [ exporter.py:   77]:	[Best Exporter]: export to ./output/best
[DEBUG] 2019-09-18 16:50:52,067 [ exporter.py:   90]:	[Best Exporter]: save inference model: "inference" to ./output/best/inference
[DEBUG] 2019-09-18 16:50:53,230 [monitored_executor.py:  122]:	saving step 1000 to ./output/model_1000
[DEBUG] 2019-09-18 16:51:04,658 [    hooks.py:  207]:	step: 1010	steps/sec: 0.02019	loss: 1.03070	lr:[6.858617e-06]
[DEBUG] 2019-09-18 16:51:14,038 [    hooks.py:  207]:	step: 1020	steps/sec: 1.03458	loss: 0.62447	lr:[6.9265243e-06]
[DEBUG] 2019-09-18 16:51:24,588 [    hooks.py:  207]:	step: 1030	steps/sec: 0.98896	loss: 0.89986	lr:[6.9944313e-06]
[DEBUG] 2019-09-18 16:51:31,941 [    hooks.py:  207]:	step: 1040	steps/sec: 1.32569	loss: 0.76850	lr:[7.062338e-06]
[DEBUG] 2019-09-18 16:51:39,364 [    hooks.py:  207]

[DEBUG] 2019-09-18 16:59:47,626 [    hooks.py:  207]:	step: 1630	steps/sec: 1.13904	loss: 0.64218	lr:[1.1068857e-05]
[DEBUG] 2019-09-18 16:59:57,491 [    hooks.py:  207]:	step: 1640	steps/sec: 1.09333	loss: 0.90164	lr:[1.1136764e-05]
[DEBUG] 2019-09-18 17:00:04,993 [    hooks.py:  207]:	step: 1650	steps/sec: 1.26889	loss: 0.59520	lr:[1.1204672e-05]
[DEBUG] 2019-09-18 17:00:13,558 [    hooks.py:  207]:	step: 1660	steps/sec: 1.09025	loss: 0.83697	lr:[1.1272578e-05]
[DEBUG] 2019-09-18 17:00:22,314 [    hooks.py:  207]:	step: 1670	steps/sec: 1.14136	loss: 1.00199	lr:[1.1340486e-05]
[DEBUG] 2019-09-18 17:00:31,719 [    hooks.py:  207]:	step: 1680	steps/sec: 1.10301	loss: 0.70694	lr:[1.1408393e-05]
[DEBUG] 2019-09-18 17:00:40,642 [    hooks.py:  207]:	step: 1690	steps/sec: 1.14856	loss: 0.69561	lr:[1.14763e-05]
[DEBUG] 2019-09-18 17:00:47,444 [    hooks.py:  207]:	step: 1700	steps/sec: 1.41233	loss: 0.66357	lr:[1.1544207e-05]
[DEBUG] 2019-09-18 17:00:57,491 [    hooks.py:  207]:	step: 1710	s

...

[DEBUG] 2019-09-19 19:26:19,453 [    hooks.py:  207]:	step: 72330	steps/sec: 1.38730	loss: 0.69783	lr:[8.8387725e-07]
[DEBUG] 2019-09-19 19:26:27,116 [    hooks.py:  207]:	step: 72340	steps/sec: 1.28468	loss: 0.07707	lr:[8.7708855e-07]
[DEBUG] 2019-09-19 19:26:36,242 [    hooks.py:  207]:	step: 72350	steps/sec: 1.12825	loss: 0.20031	lr:[8.7029633e-07]
[DEBUG] 2019-09-19 19:26:44,499 [    hooks.py:  207]:	step: 72360	steps/sec: 1.23824	loss: 0.22666	lr:[8.6350457e-07]
[DEBUG] 2019-09-19 19:26:50,982 [    hooks.py:  207]:	step: 72370	steps/sec: 1.54348	loss: 0.16996	lr:[8.567157e-07]
[DEBUG] 2019-09-19 19:26:57,792 [    hooks.py:  207]:	step: 72380	steps/sec: 1.37465	loss: 0.29380	lr:[8.4992365e-07]
[DEBUG] 2019-09-19 19:27:07,275 [    hooks.py:  207]:	step: 72390	steps/sec: 1.09511	loss: 0.09915	lr:[8.431346e-07]
[DEBUG] 2019-09-19 19:27:15,940 [    hooks.py:  207]:	step: 72400	steps/sec: 1.20712	loss: 0.15985	lr:[8.363426e-07]
[DEBUG] 2019-09-19 19:27:25,155 [    hooks.py:  207]:	step:

[DEBUG] 2019-09-19 19:35:26,828 [    hooks.py:  207]:	step: 72980	steps/sec: 1.45911	loss: 0.25368	lr:[4.4248998e-07]
[DEBUG] 2019-09-19 19:35:36,166 [    hooks.py:  207]:	step: 72990	steps/sec: 1.14862	loss: 0.34600	lr:[4.3570108e-07]
[DEBUG] 2019-09-19 19:35:42,584 [    hooks.py:  207]:	step: 73000	steps/sec: 1.46793	loss: 0.41725	lr:[4.289091e-07]
[DEBUG] 2019-09-19 19:35:42,585 [monitored_executor.py:  281]:	freezing program
[DEBUG] 2019-09-19 19:35:42,585 [monitored_executor.py:  251]:	will not freeze a program without loss
[DEBUG] 2019-09-19 19:35:42,585 [monitored_executor.py:  283]:	done freezing
[INFO] 2019-09-19 19:35:42,585 [monitored_executor.py:  284]:	********** Start Loop ************
[DEBUG] 2019-09-19 19:35:42,585 [monitored_executor.py:  289]:	train loop has hook <propeller.paddle.train.hooks.StopAtStepHook object at 0x7fbf60615f28>
[DEBUG] 2019-09-19 19:35:42,585 [monitored_executor.py:  289]:	train loop has hook <propeller.paddle.train.hooks.EvalHook object at 0x7fb

[DEBUG] 2019-09-19 19:50:06,399 [    hooks.py:  207]:	step: 73440	steps/sec: 1.20901	loss: 0.37308	lr:[1.3012594e-07]
[DEBUG] 2019-09-19 19:50:15,982 [    hooks.py:  207]:	step: 73450	steps/sec: 1.10159	loss: 0.57594	lr:[1.2333399e-07]
[DEBUG] 2019-09-19 19:50:24,809 [    hooks.py:  207]:	step: 73460	steps/sec: 1.12811	loss: 0.19768	lr:[1.1654502e-07]
[DEBUG] 2019-09-19 19:50:32,882 [    hooks.py:  207]:	step: 73470	steps/sec: 1.17914	loss: 0.25062	lr:[1.0975307e-07]
[DEBUG] 2019-09-19 19:50:41,837 [    hooks.py:  207]:	step: 73480	steps/sec: 1.10132	loss: 0.13024	lr:[1.0296407e-07]
[DEBUG] 2019-09-19 19:50:51,182 [    hooks.py:  207]:	step: 73490	steps/sec: 1.10918	loss: 0.30766	lr:[9.61721e-08]
[DEBUG] 2019-09-19 19:51:00,698 [    hooks.py:  207]:	step: 73500	steps/sec: 1.07685	loss: 0.11769	lr:[8.938315e-08]
[DEBUG] 2019-09-19 19:51:07,604 [    hooks.py:  207]:	step: 73510	steps/sec: 1.41962	loss: 0.33494	lr:[8.259121e-08]
[DEBUG] 2019-09-19 19:51:16,239 [    hooks.py:  207]:	step: 

[INFO] 2019-09-19 19:52:46,398 [    hooks.py:  219]:	StopAtStepHook called stop
[INFO] 2019-09-19 19:52:46,399 [monitored_executor.py:  336]:	********** Stop Loop ************


[None, None, None, None, None]

## Predict

In [15]:
# after training you might want to check your model performance on test-set
# let's do this via `propeller.predict`
# keep in mind that model of best performace has been exported during thet `train_and_eval` phrase

best_filename = [file for file in os.listdir(os.path.join(OUTPUT_DIR, 'best_model')) if 'model' in file][0]
best_model_path = os.path.join(os.path.join(OUTPUT_DIR, 'best_model'), best_filename)
true_label = [label_map[(line.strip().split(b'\t')[-1])]for line in open('./xnli_data/test/part.0', 'rb')]

def drop_label(sentence, segments, label): #we drop the label column here
    return sentence, segments

test_ds = feature_column.build_dataset('test', use_gz=False, data_dir='./xnli_data/test', shuffle=False, repeat=False) \
                               .map(before_pad) \
                               .padded_batch(BATCH, (0, 0, 0)) \
                               .map(after_pad) \
                               .map(drop_label)



[INFO] 2019-09-19 19:52:46,983 [feature_column.py:  266]:	reading raw files from ./xnli_data/test/part.0


In [16]:
result = []
learner = propeller.Learner(ClassificationErnieModel, run_config, params=config, )
for pred in learner.predict(test_ds, ckpt=-1):
    result.append(np.argmax(pred))
    
result, true_label = np.array(result), np.array(true_label)

test_acc = (result == true_label).sum() / len(true_label)
print('test accuracy:%.5f' % test_acc)

[INFO] 2019-09-19 19:52:46,992 [  trainer.py:  237]:	Building Predict Graph
[INFO] 2019-09-19 19:52:46,992 [functional.py:  313]:	Try to infer data shapes & types from generator
[INFO] 2019-09-19 19:52:47,004 [functional.py:  326]:	Dataset `predict` has data_shapes: [(16, 82, 1), (16, 82, 1)] data_types: ['int64', 'int64']
[INFO] 2019-09-19 19:52:47,577 [  trainer.py:  241]:	Done
[INFO] 2019-09-19 19:52:47,613 [  trainer.py:  247]:	Predict with: 
> Run_config: RunConfig(model_dir='./output', run_steps=None, max_steps=73631.625, save_steps=1000, eval_steps=1000, eval_max_steps=None, skip_steps=10, log_steps=10, max_ckpt=3, shit=None)
> Params: {'attention_probs_dropout_prob': 0.1, 'hidden_act': 'relu', 'hidden_dropout_prob': 0.1, 'hidden_size': 768, 'initializer_range': 0.02, 'max_position_embeddings': 513, 'num_attention_heads': 12, 'num_hidden_layers': 12, 'type_vocab_size': 2, 'vocab_size': 18000, 'sent_type_vocab_size': None, 'use_task_id': False, 'task_id': 0, 'warmup_proportion': 

test accuracy:0.7780439121756487


[INFO] 2019-09-19 20:08:45,357 [monitored_executor.py:  336]:	********** Stop Loop ************


## Serving
your model is now ready to serve! 
you can open up a server by propeller with 
```script
python -m propeller.tools.start_server -m /path/to/saved/model -p 8888
```
